In [25]:
import dash
from dash.dependencies import Input, Output, State
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import dash_table_experiments as dt
import plotly
from IPython import display
import json
from collections import OrderedDict

In [28]:
from IPython import display
def show_app(app,  # type: dash.Dash
             port=8080,
             width=1300,
             height=500,
             offline=True,
             style=True,
             **dash_flask_kwargs):
    """
    Run the application inside a Jupyter notebook and show an iframe with it
    :param app:
    :param port:
    :param width:
    :param height:
    :param offline:
    :return:
    """
    url = 'http://localhost:%d' % port
    iframe = '<iframe src="{url}" width={width} height={height}></iframe>'.format(url=url,
                                                                                  width=width,
                                                                                  height=height)
    display.display_html(iframe, raw=True)
    if offline:
        app.css.config.serve_locally = True
        app.scripts.config.serve_locally = True
    if style:
        external_css = ["https://fonts.googleapis.com/css?family=Raleway:400,300,600",
                        "https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css",
                        "http://getbootstrap.com/dist/css/bootstrap.min.css", ]

        for css in external_css:
            app.css.append_css({"external_url": css})

        external_js = ["https://code.jquery.com/jquery-3.2.1.min.js",
                       "https://cdn.rawgit.com/plotly/dash-app-stylesheets/a3401de132a6d0b652ba11548736b1d1e80aa10d/dash-goldman-sachs-report-js.js",
                       "http://getbootstrap.com/dist/js/bootstrap.min.js"]

        for js in external_js:
            app.scripts.append_script({"external_url": js})

    return app.run_server(debug=False,
                          port=port,
                          **dash_flask_kwargs)

In [30]:
file = "Desktop/Market Analysis/SSIS_BSC_ISIC4_21042020232635595.csv"
df=pd.read_csv(file,header=0)
df.rename(columns={'ISIC4.1':'Industry',},inplace=True)

In [32]:
df1= df[['Country','Industry','Source','Size Class','Time','Value']]

In [34]:
df1=df1.loc[df1['Source']=='Business Statistics by Employment Size Class']

In [36]:
df2=df1.loc[df1['Time']==2017]

In [37]:
df2=df2.groupby(['Country','Size Class','Industry']).agg('sum')

In [38]:
df2

Time  \
Country        Size Class                                Industry                                                    
Australia      1-249 persons employed (SMEs)             Accommodation and food service activities            2017   
                                                         Administrative and support service activities        2017   
                                                         Business economy, except financial and insuranc...   2017   
                                                         Construction                                         2017   
                                                         Electricity, gas, steam and air conditioning su...   2017   
                                                         Industry (except construction)                       2017   
                                                         Information and communication                        2017   
                                                         Manufacturing                                        2017   
                                                         Professional, scientific and technical activities    2017   
                                                         Real estate activities                               2017   
                                                         Transportation and storage                           2017   
                                                         Water supply; sewerage, waste management and re...   2017   
                                                         Wholesale and retail trade; repair of motor veh...   2017   
               250+ persons employed (Large enterprises) Accommodation and food service activities            2017   
                                                         Administrative and support service activities        2017   
                                                         Business economy, except financial and insuranc...   2017   
                                                         Construction                                         2017   
                                                         Electricity, gas, steam and air conditioning su...   2017   
                                                         Industry (except construction)                       2017   
                                                         Information and communication                        2017   
                                                         Manufacturing                                        2017   
                                                         Professional, scientific and technical activities    2017   
                                                         Real estate activities                               2017   
                                                         Transportation and storage                           2017   
                                                         Water supply; sewerage, waste management and re...   2017   
                                                         Wholesale and retail trade; repair of motor veh...   2017   
               Total                                     Accommodation and food service activities            2017   
                                                         Administrative and support service activities        2017   
                                                         Business economy, except financial and insuranc...   2017   
                                                         Construction                                         2017   
...                                                                                                            ...   
United Kingdom 1-249 persons employed (SMEs)             Real estate activities                               8068   
                                                         Transportation and storage               

In [39]:
df2=df2.reset_index()
df2.columns

Index([u'Country', u'Size Class', u'Industry', u'Time', u'Value'], dtype='object')

In [ ]:
def generate_table(dataframe, max_rows=10):
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns])] +

        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(min(len(dataframe), max_rows))]
    )

app = dash.Dash()

app.layout = html.Div(children=[
    html.H4(children='US Agriculture Exports (2011)'),
    dcc.Dropdown(id='dropdown', options=[
        {'label': i, 'value': i} for i in df2.Industry.unique()
    ], multi=True, placeholder='Filter by industry...'),
    html.Div(id='table-container')
])

@app.callback(
    dash.dependencies.Output('table-container', 'children'),
    [dash.dependencies.Input('dropdown', 'value')])
def display_table(dropdown_value):
    if dropdown_value is None:
        return generate_table(df2)

    dff = df2[df2.Industry.str.contains('|'.join(dropdown_value))]
    return generate_table(dff)

app.css.append_css({"external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"})


if __name__ == '__main__':
    app.run_server(debug=False)

In [ ]:
show_app(app)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [03/May/2020 04:29:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 04:29:42] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 04:29:43] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 04:29:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 04:30:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 04:30:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 04:30:06] "POST /_dash-update-component HTTP/1.1" 200 -
